In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# For optimization
from scipy.optimize import minimize

plt.rcParams['figure.figsize'] = (12,6)
plt.rcParams['axes.grid'] = True

# Paths
DATA_DIR = "../data"

# Files from Task 3
TSLA_FC_FILE = f"{DATA_DIR}/tsla_forecast_arima.csv"  # or LSTM CSV if that's your best model


In [ ]:
# Load cleaned historical data (BND, SPY, TSLA)
close_prices_clean = pd.read_csv(f"{DATA_DIR}/close_prices_clean.csv", index_col=0, parse_dates=True)

# Extract only the 3 assets
cols_needed = []
for asset in ["BND", "SPY", "TSLA"]:
    for c in close_prices_clean.columns:
        if asset in c:
            cols_needed.append(c)
            break
hist_prices = close_prices_clean[cols_needed].dropna().sort_index()
hist_prices.columns = ["BND", "SPY", "TSLA"]

# Load your TSLA forecast from best model (e.g., ARIMA output)
tsla_fc = pd.read_csv(TSLA_FC_FILE, index_col=0, parse_dates=True)


In [ ]:
# Historical daily returns
daily_returns_hist = hist_prices.pct_change().dropna()

# Annualization factor for daily data (trading days)
TRADING_DAYS = 252

# Expected returns vector
# TSLA: from forecast
tsla_daily_ret_fc = tsla_fc["ARIMA_Forecast"].pct_change().dropna()
tsla_exp_annual_ret = (1 + tsla_daily_ret_fc.mean())**TRADING_DAYS - 1

# BND, SPY: from history
bnd_exp_annual_ret = (1 + daily_returns_hist["BND"].mean())**TRADING_DAYS - 1
spy_exp_annual_ret = (1 + daily_returns_hist["SPY"].mean())**TRADING_DAYS - 1

exp_returns = np.array([bnd_exp_annual_ret, spy_exp_annual_ret, tsla_exp_annual_ret])
assets = ["BND", "SPY", "TSLA"]

print("Expected Annual Returns:")
for a, r in zip(assets, exp_returns):
    print(f"{a}: {r:.2%}")


Covariance matrix (historical)

In [ ]:
# Annualized covariance matrix from daily returns
cov_matrix_annual = daily_returns_hist.cov() * TRADING_DAYS
cov_matrix_annual


Portfolio functions

In [ ]:
def portfolio_performance(weights, exp_returns, cov_matrix):
    port_return = np.dot(weights, exp_returns)
    port_vol    = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return port_return, port_vol

def neg_sharpe_ratio(weights, exp_returns, cov_matrix, risk_free_rate=0.02):
    p_ret, p_vol = portfolio_performance(weights, exp_returns, cov_matrix)
    return -(p_ret - risk_free_rate) / p_vol

def min_volatility(weights, exp_returns, cov_matrix):
    return portfolio_performance(weights, exp_returns, cov_matrix)[1]


Optimization setup

In [ ]:
# Constraints: sum(weights) == 1
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})
bounds = tuple((0,1) for _ in assets)
init_guess = np.array([1/3]*3)

# Max Sharpe (Tangency Portfolio)
opt_sharpe = minimize(neg_sharpe_ratio, init_guess,
                      args=(exp_returns, cov_matrix_annual, 0.02),
                      method='SLSQP', bounds=bounds, constraints=constraints)

# Min Volatility Portfolio
opt_minvol = minimize(min_volatility, init_guess,
                      args=(exp_returns, cov_matrix_annual),
                      method='SLSQP', bounds=bounds, constraints=constraints)

w_sharpe = opt_sharpe.x
w_minvol = opt_minvol.x


Build the Efficient Frontier

In [ ]:
plt.plot(frontier_vols, target_returns, 'g--', label='Efficient Frontier')

# Mark max Sharpe
sharpe_ret, sharpe_vol = portfolio_performance(w_sharpe, exp_returns, cov_matrix_annual)
plt.scatter(sharpe_vol, sharpe_ret, c='red', marker='*', s=200, label='Max Sharpe')

# Mark min vol
minvol_ret, minvol_vol = portfolio_performance(w_minvol, exp_returns, cov_matrix_annual)
plt.scatter(minvol_vol, minvol_ret, c='blue', marker='*', s=200, label='Min Volatility')

plt.title("Efficient Frontier – TSLA, BND, SPY")
plt.xlabel("Volatility (Std. Dev.)")
plt.ylabel("Expected Return")
plt.legend()
plt.show()


Final recommended portfolio

In [ ]:
# Choose: Max Sharpe
opt_weights = w_sharpe
opt_ret, opt_vol = portfolio_performance(opt_weights, exp_returns, cov_matrix_annual)
opt_sharpe_ratio = (opt_ret - 0.02) / opt_vol

print("=== Recommended Portfolio (Max Sharpe) ===")
for a, w in zip(assets, opt_weights):
    print(f"{a}: {w:.2%}")
print(f"Expected Annual Return: {opt_ret:.2%}")
print(f"Annual Volatility: {opt_vol:.2%}")
print(f"Sharpe Ratio (Rf=2%): {opt_sharpe_ratio:.2f}")
